In [2]:
%pwd

'C:\\Users\\hsy02'

In [4]:
%cd .\yolov5

C:\Users\hsy02\yolov5


In [6]:
!pip install Pillow==10.3

In [7]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [9]:
import os
import shutil
from sklearn.model_selection import train_test_split

def create_validation_set(train_path, val_path, split_ratio=0.3):
    """
    Train 데이터의 일부를 Val로 이동
    """
    # 필요한 디렉토리 생성
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    # Train 이미지 리스트 가져오기
    train_images = os.listdir(os.path.join(train_path, 'images'))
    train_images = [f for f in train_images if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Train/Val 분할
    _, val_images = train_test_split(train_images,
                                   test_size=split_ratio,
                                   random_state=42)

    # Val로 파일 복사
    for image_file in val_images:
        # 이미지 복사
        src_image = os.path.join(train_path, 'images', image_file)
        dst_image = os.path.join(val_path, 'images', image_file)
        shutil.copy2(src_image, dst_image)

        # 라벨 파일 복사
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_label = os.path.join(train_path, 'labels', label_file)
        dst_label = os.path.join(val_path, 'labels', label_file)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"Created validation set with {len(val_images)} images")

# 실행
train_path = r'C:\Users\hsy02\yolov5\Train'
val_path = r'C:\Users\hsy02\yolov5\Val'

create_validation_set(train_path, val_path)

Created validation set with 705 images


In [12]:
def check_dataset():
    train_path = r'C:\Users\hsy02\yolov5\Train'
    val_path = r'C:\Users\hsy02\yolov5\Val'

    # Train 데이터 확인
    train_images = len(os.listdir(os.path.join(train_path, 'images')))
    train_labels = len(os.listdir(os.path.join(train_path, 'labels')))

    # Val 데이터 확인
    val_images = len(os.listdir(os.path.join(val_path, 'images')))
    val_labels = len(os.listdir(os.path.join(val_path, 'labels')))

    print("Dataset status:")
    print(f"Train - Images: {train_images}, Labels: {train_labels}")
    print(f"Val - Images: {val_images}, Labels: {val_labels}")

# 데이터셋 상태 확인
check_dataset()

Dataset status:
Train - Images: 2347, Labels: 2347
Val - Images: 705, Labels: 705


In [14]:
#학습시작

In [17]:
!pip install torch

In [18]:
#필요 라이브러리 임포트하기
import torch
import os
from IPython.display import Image, clear_output # to display images

In [19]:
!pip install tensorflow

In [21]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.eager.context import eager_mode

def _preproc(image, output_height=512, output_width=512, resize_side=512):
    ''' imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px'''
    with eager_mode():
        h, w = image.shape[0], image.shape[1]
        scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
        resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h*scale), int(w*scale)])
        cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)
        return tf.squeeze(cropped_image)

def Create_npy(imagespath, imgsize, ext) :
    images_list = [img_name for img_name in os.listdir(imagespath) if
                os.path.splitext(img_name)[1].lower() == '.'+ext.lower()]
    calib_dataset = np.zeros((len(images_list), imgsize, imgsize, 3), dtype=np.float32)

    for idx, img_name in enumerate(sorted(images_list)):
        img_path = os.path.join(imagespath, img_name)
        try:
            # 파일 크기가 정상적인지 확인
            if os.path.getsize(img_path) == 0:
                print(f"Error: {img_path} is empty.")
                continue

            img = Image.open(img_path)
            img = img.convert("RGB")  # RGBA 이미지 등 다른 형식이 있을 경우 강제로 RGB로 변환
            img_np = np.array(img)

            img_preproc = _preproc(img_np, imgsize, imgsize, imgsize)
            calib_dataset[idx,:,:,:] = img_preproc.numpy().astype(np.uint8)
            print(f"Processed image {img_path}")

        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    np.save('calib_set.npy', calib_dataset)


In [22]:
# "cannot identify image file" 에러가 발생하는 경우, PILLOW Version을 "!pip install Pillow==10.1" 로 변경하여 설치합니다.
Create_npy(r'C:\Users\hsy02\yolov5\Train\images', 512, 'jpg')

In [34]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.1/38.8 MB 1.7 MB/s eta 0:00:23
   ---------------------------------------- 0.3/38.8 MB 2.2 MB/s eta 0:00:18
    --------------------------------------- 0.6/38.8 MB 3.4 MB/s eta 0:00:12
    --------------------------------------- 0.8/38.8 MB 3.8 MB/s eta 0:00:11
   - -------------------------------------- 1.1/38.8 MB 4.4 MB/s eta 0:00:09
   - -------------------------------------- 1.3/38.8 MB 4.2 MB/s eta 0:00:09
   - -------------------------------------- 1.6/38.8 MB 4.7 MB/s eta 0:00:08
   - -------------------------------------- 1.9/38.8 MB 4.9 MB/s eta 0:00:08
   -- ------------------------------------- 2.2/38.8 MB 4.9 MB/s eta 0:00:08
   -- ------------------------------------- 2.5/38.8 MB 5.0 MB/s eta 0:00:08
   -- ------------------------------------- 2.8/38.8 MB 5.3 MB/s eta 0:00:07
   ---

In [25]:
%pwd

'C:\\Users\\hsy02\\yolov5'

In [64]:
!python C:\Users\hsy02\yolov5\train.py --img 512 --batch 16 --epochs 300 --data C:\Users\hsy02\yolov5\data.yaml --weights ./yolov5/yolov5n.pt --cache

^C


In [28]:
import subprocess

command = ["python", "C:/Users/hsy02/yolov5/train.py", "--img", "512", "--batch", "16", "--epochs", "50", "--data", "C:/Users/hsy02/yolov5/data.yaml", "--weights", "./yolov5/yolov5n.pt", "--cache"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end="")  # 실시간으로 출력


train: weights=./yolov5/yolov5n.pt, cfg=, data=C:/Users/hsy02/yolov5/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-383-g1435a8ee Python-3.12.4 torch-2.5.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1


KeyboardInterrupt

